# DAC Contest
This reference design will help you walk through a design flow of DAC SDC 2023. This is a simplified design to help users get started on the FPGA platform and to understand the overall flow. It does not contain any object detection hardware.

If you have any questions, please post on the Slack page (link on SDC website sidebar).

### Hardware
The base hardware design contains the Zynq MPSoC processor (A53) with a DMA and FIFO implemented in the PL.
Note that in this starting hardware, no actual image processing/detection is done. Pictures are:
  * Sent from PS to DMA, then DMA to FIFO
  * Sent back from FIFO to DMA, then DMA to PS.

Note that the FIFO should be replaced with meaningful hardware to process the pictures.

You can create a Vivado project by opening Vivado 2022.2, changing to the hw directory, and running `make`. This will create and open a Vivado project that you can edit.  On the final submission you are required to submit the sources files of your design, and you should run `File->Project->Write Tcl...` to generate a new tcl file to submit. Be sure to check the `Recreate Block Designs using Tcl` box, and include any custom IP modules you use.

### Software
Note:
  * You will not submit your `dac_sdc.py` file, so any changes you make to this file will not be considered during evluation.  
  * You can use both PS and PL side to do inference.

### Object Detection

Object detection will be done on images in batches:
  * You will provide a Python callback function that will perform object detection on batch of images.  This callback function wile be called many times.
  * The callback function should return the locations of all images in the batch.
  * Runtime will be recorded during your callback function.
  * Images will be loaded from SD card before each batch is run, and this does not count toward your energy usage or runtime.
  
### Notebook
Your notebook should contain 4 code cells:

1. Importing all libraries and creating your Team object.
1. Downloading the overlay, and performany any one-time configuration.
1. Python callback function and any other Python helper functions.
1. Running object detection
1. Cleanup



## 1. Imports and Create Team

In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../common"))

import math
import time
import numpy as np
from PIL import Image
from matplotlib import pyplot
import cv2
from datetime import datetime

import pynq
import dac_sdc
from IPython.display import display

team_name = 'sample_team'
dac_sdc.BATCH_SIZE = 2
team = dac_sdc.Team(team_name)

**Your team directory where you can access your bitstream, notebook, and any other files you submit, is available as `team.team_dir`.**

## 2. Preparing the overlay/bitstream and weight loading
Overlay/bitstream loading must be executed in this cell.

In this sample hardware, the DMA instance is exposed as an attribute of the overlay object.  You aren't required to use DMA for your hardware solution.

In [ ]:
bitfile = team.get_bitstream_path()
print(bitfile)
overlay = pynq.Overlay(bitfile)
dma = overlay.axi_dma_0

## 3. Python Callback Function and Helper Functions


### Pushing the picture through the pipeline
In this example, we use contiguous memory arrays for sending and receiving data via DMA.

The size of the buffer depends on the size of the input or output data.  The example images are 640x360 (same size as training and test data), and we will use `pynq.allocate` to allocate contiguous memory.

### Callback function
The callback function:
  - Will be called on each batch of images (will be called many times)
  - Is prvided with a list of tuples of (image path, RGB image)
  - It should return a dictionary with an entry for each image:
    - Key: Image name (`img_path.name`)
    - Value: Dictionary of item type and bounding box (keys: `type`, `x`, `y`, `width`, `height`)

See the code below for an example:


In [ ]:
in_buffer = pynq.allocate(shape=(180, 320, 3), dtype=np.uint8, cacheable = 1)
out_buffer = pynq.allocate(shape=(180, 320, 3), dtype=np.uint8, cacheable = 1)

def dma_transfer():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)    
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
def my_callback(rgb_imgs):
    object_locations_by_image = {}
    
    
    for (img_path, img) in rgb_imgs:
        object_locations = []
        
        # Resize the image (this is part of your runtime)
        img_resized = cv2.resize(img, (320,180), interpolation = cv2.INTER_AREA)
        
        # Put resized image into DMA
        print("Loading image", img_path.name, "into buffer for DMA transfer")
        in_buffer[:] = img_resized    
        dma_transfer()
        del img_resized
        print("\tDMA transfer complete")
        
        # Show image (notebook will only show latest image)
        pyplot.imshow(out_buffer)
        
        # Appending fake image location, since this example doesn't actually perform object detection 
        object_locations.append({"type":1, "x":1168, "y":639, "width":457, "height":245})
        object_locations.append({"type":1, "x":831, "y":626, "width":77, "height":57})
        object_locations.append({"type":2, "x":753, "y":628, "width":44, "height":52})
        
        # Save to dictionary by image filename
        object_locations_by_image[img_path.name] = object_locations
        
    return object_locations_by_image


## 4. Running Object Detection

Call the following function to run the object detection.  Extra debug output is enabled when `debug` is `True`.

In [ ]:
team.run(my_callback, debug=True)

## 5. Cleanup

In [ ]:
# Remember to free the contiguous memory after usage.
del in_buffer
del out_buffer